In [24]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [29]:
# Read the text from the file
with open('dataset.txt', 'r', encoding='utf-8') as file:
    faqs = file.read()

In [30]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([faqs])
len(tokenizer.word_index)

1088

In [31]:
#tokenizer.word_index

In [32]:
for sentence in faqs.split('\n'):
  print(tokenizer.texts_to_sequences([sentence]))

[[567, 568]]
[[387, 299, 35, 569, 12, 388, 24, 48, 1, 193]]
[[570, 102, 571, 89, 572, 573, 574, 1, 238, 575]]
[[576, 102, 577, 389, 390, 578, 194, 579, 580, 6, 581, 239]]
[[391, 392, 582, 76, 583, 393, 584, 392]]
[[585, 394, 195, 586, 8, 587, 588, 18, 103, 589]]
[[]]
[[12, 590, 300, 35, 591, 1, 592, 77, 240, 593, 594]]
[[595, 596, 395, 301, 597, 6, 241, 598, 38, 12, 599, 600, 601]]
[[602, 395, 89, 242, 241, 603, 1, 604, 12, 396, 605, 606]]
[[607, 608, 8, 397, 609, 610, 1, 611, 77, 78]]
[[12, 612, 35, 613, 6, 614, 79, 300]]
[[]]
[[398, 160, 615, 1, 12, 616, 399, 8, 238, 617, 618]]
[[302, 78, 243, 8, 619, 303, 12, 161, 620, 103, 12, 400]]
[[621, 622, 401, 12, 623, 624, 244, 402, 244]]
[[238, 625, 8, 626, 12, 627, 628, 78, 629, 630]]
[[403, 631, 117, 304, 1, 632, 12, 633, 404, 245]]
[[]]
[[405, 35, 246, 196, 406, 38, 12, 634, 104, 407]]
[[635, 35, 305, 24, 12, 408, 636, 405, 637, 300]]
[[78, 301, 80, 638, 639, 409, 8, 197, 117, 2]]
[[640, 76, 78, 641, 1, 642, 306, 38, 117, 643]]
[[644, 90